In [284]:
import pandas as pd
import numpy as np
from functools import reduce

## NOAA Daily Weather Data

### Description: 

Daily weather data for three cities (New York, Chicago, and Los Angeles) between Jan 1, 2015 and Dec 31, 2018.

[Data Source](https://www.ncdc.noaa.gov/cdo-web/search?datasetid=GHCND)

### Metadata:

*All units are metric!*

[Metadata documentation source](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf)

* Date Local: Date in which weather record is valid
* City Name: City in which weather record is valid
* PRCP: Daily Precipitation (millimeters)
* AWND: Average Daily Wind Speed (meters/second)
* SNWD: Snow depth (millimeters)
* SNOW: Snowfall (millimeters)
* TMAX = Maximum temperature (Celsius)
* TMIN = Minimum temperature (Celsius)

In [296]:
# Chicago
CHI_noaa_2015 = pd.read_csv('NOAA/CHI/CHI_2015.csv')
CHI_noaa_2017 = pd.read_csv('NOAA/CHI/CHI_2017.csv')
CHI_2018 = pd.read_csv('NOAA/CHI/CHI_2018.csv')

# Los Angeles
LA_noaa_2015 = pd.read_csv('NOAA/LA/LA_2015.csv')
LA_noaa_2017 = pd.read_csv('NOAA/LA/LA_2017.csv')
LA_noaa_2018 = pd.read_csv('NOAA/LA/LA_2018.csv')

# New York City
NYC_noaa_2015 = pd.read_csv('NOAA/NYC/NYC_2015.csv')
NYC_noaa_2017 = pd.read_csv('NOAA/NYC/NYC_2017.csv')
NYC_noaa_2018 = pd.read_csv('NOAA/NYC/NYC_2018.csv')

CHI_noaa = pd.concat([CHI_noaa_2015,CHI_noaa_2017,CHI_2018],sort=False)
LA_noaa = pd.concat([LA_noaa_2015,LA_noaa_2017,LA_noaa_2018],sort=False)
NYC_noaa = pd.concat([NYC_noaa_2015,NYC_noaa_2017,NYC_noaa_2018],sort=False)

# Add new columns with City Name (to map with EPA data below)
CHI_noaa['City Name'] = 'Chicago'
LA_noaa['City Name'] = 'Los Angeles'
NYC_noaa['City Name'] = 'New York'

# Concatenate city weather data into single dataframe
noaa_DF = pd.concat([CHI_noaa,LA_noaa,NYC_noaa],sort=False)

# Discard certain attributes
noaa_DF = noaa_DF[['DATE','City Name','PRCP','AWND','SNWD','SNOW','TMAX','TMIN']]

# Rename 'DATE' to 'Date Local' for inner joining noaa and epa data
noaa_DF.rename(columns={'DATE': 'Date Local'}, inplace=True)

In [286]:
noaa_DF.head(5)

,Date Local,City Name,PRCP,AWND,SNWD,SNOW,TMAX,TMIN
0,2015-03-25,Chicago,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-03-26,Chicago,0.0,NaN,NaN,0.0,NaN,NaN
2,2015-03-27,Chicago,1.5,NaN,NaN,NaN,NaN,NaN
3,2015-03-28,Chicago,0.0,NaN,NaN,0.0,NaN,NaN
4,2015-03-29,Chicago,0.0,NaN,NaN,0.0,NaN,NaN


## EPA AIR QUALITY

### Description:

Daily air quality data for three cities (New York, Chicago, and Los Angeles) between Jan 1, 2015 and Dec 31, 2018.

[Data Source](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

### Metadata:

*All units are metric!*

[Metadata documentation source](https://aqs.epa.gov/aqsweb/airdata/FileFormats.html)

* Date Local: Date in which weather record is valid
* City Name: City in which weather record is valid
* Event Type: Whether exceptional event (i.e. wildfire) occured during data period data was collected
* 1st Max Value: Highest value for the day for given pollutant
* AQI: Air Quality Index for given pollutant
* Arithmetic Mean: Average value for the day for given pollutant

The pollutants included in the dataset (and their respective units of measurement) are provided below:
* CO: parts per million
* SO2: parts per billion
* Ozone: parts per million
* NO2: parts per billion

In [287]:
def load_pollutant_datasets(folder_path,data_type,years):
    aq_data = [] 
    for year in years:
        df = pd.read_csv('{}/{}_{}.csv'.format(folder_path,data_type,year))
        aq_data.append(df)
       
    aq_dataFrame = pd.concat(aq_data)
    return aq_dataFrame

folder_path = 'EPA'
years = [2015,2016,2017,2018]
co_dataFrame = load_epa_data(folder_path,'CO',years)
so2_dataFrame = load_epa_data(folder_path,'SO2',years)
ozone_dataFrame = load_epa_data(folder_path,'Ozone',years)
no2_dataFrame = load_epa_data(folder_path,'NO2',years)

In [288]:
def filter_pollutant_datasets(dataframe,pollutant_type):
    # Default for all datasets except Ozone
    sample_duration = '1 HOUR'
    
    # Rename all instances of East Chicago to Chicago
    dataframe['City Name'] = dataframe['City Name'].replace({'East Chicago': 'Chicago'})
    
    # Handle special case: Chicago monitor site not the same across datasets; cannot be hardcoded
    chicago_site_num = 76
    
    if pollutant_type == "Ozone":
        sample_duration = '8-HR RUN AVG BEGIN HOUR'
    if pollutant_type == "CO":
        chicago_site_num = 15
        
    df1 = dataframe[(dataframe['City Name'] == 'New York') & (dataframe['Sample Duration'] == sample_duration) & (dataframe['Site Num'] == 133)]
    df2 = dataframe[(dataframe['City Name'] == 'Chicago') & (dataframe['Sample Duration'] == sample_duration) & (dataframe['Site Num'] == chicago_site_num)]
    df3 = dataframe[(dataframe['City Name'] == 'Los Angeles') & (dataframe['Sample Duration'] == sample_duration) & (dataframe['Site Num'] == 5005)]
    dataframe_ = df1.append([df2,df3])
    
    # Choose only important columns for analysis
    dataframe = dataframe_[['Date Local','City Name','Event Type','1st Max Value','AQI','Arithmetic Mean']]
     
    # Add pollutant prefix to every column EXCEPT Local Date and City Name
    cols = dataframe.columns[~dataframe.columns.str.contains('City Name|Date Local')]
    suffix = '_{}'.format(pollutant_type)
    dataframe = dataframe.rename(columns = dict(zip(cols, cols + suffix)))
    return dataframe

co_dataFrame_ = filter_pollutant_datasets(co_dataFrame,"CO")
so2_dataFrame_ = filter_pollutant_datasets(so2_dataFrame,"SO2")
ozone_dataFrame_ = filter_pollutant_datasets(ozone_dataFrame,"Ozone")
no2_dataFrame_ = filter_pollutant_datasets(no2_dataFrame,"NO2")

In [289]:
def merge_epa_dataframes(dfs):
    df_final = reduce(lambda left,right: pd.merge(left,right,on=["Date Local","City Name"],how="inner"), dfs)
    return df_final

epa_data = merge_epa_dataframes([co_dataFrame_,so2_dataFrame_,ozone_dataFrame_,no2_dataFrame_])
len(epa_data)

4013

In [290]:
epa_data.groupby(['City Name']).count()

,Date Local,Event Type_CO,1st Max Value_CO,AQI_CO,Arithmetic Mean_CO,Event Type_SO2,1st Max Value_SO2,AQI_SO2,Arithmetic Mean_SO2,Event Type_Ozone,1st Max Value_Ozone,AQI_Ozone,Arithmetic Mean_Ozone,Event Type_NO2,1st Max Value_NO2,AQI_NO2,Arithmetic Mean_NO2
City Name,,,,,,,,,,,,,,,,,
Chicago,1250,1250,1250,0,1250,1250,1250,1250,1250,1250,1250,1250,1250,1250,1250,1250,1250
Los Angeles,1330,1330,1330,0,1330,1330,1330,1330,1330,1330,1330,1330,1330,1330,1330,1330,1330
New York,1433,1433,1433,0,1433,1433,1433,1433,1433,1433,1433,1433,1433,1433,1433,1433,1433


# Merge and Serialize Datasets

In [291]:
epa_noaa_df = pd.merge(epa_data,noaa_DF,on=["Date Local","City Name"],how="inner")

In [292]:
epa_noaa_df.head(5)

,Date Local,City Name,Event Type_CO,1st Max Value_CO,AQI_CO,Arithmetic Mean_CO,Event Type_SO2,1st Max Value_SO2,AQI_SO2,Arithmetic Mean_SO2,...,Event Type_NO2,1st Max Value_NO2,AQI_NO2,Arithmetic Mean_NO2,PRCP,AWND,SNWD,SNOW,TMAX,TMIN
0,2015-01-01,New York,None,0.4,NaN,0.2875,None,5.3,7.0,3.0625,...,None,29.8,27,20.0625,0.0,NaN,NaN,0.0,NaN,NaN
1,2015-01-01,New York,None,0.4,NaN,0.2875,None,5.3,7.0,3.0625,...,None,29.8,27,20.0625,0.0,NaN,0.0,0.0,-0.6,-10.0
2,2015-01-01,New York,None,0.4,NaN,0.2875,None,5.3,7.0,3.0625,...,None,29.8,27,20.0625,0.0,NaN,0.0,0.0,NaN,NaN
3,2015-01-01,New York,None,0.4,NaN,0.2875,None,5.3,7.0,3.0625,...,None,29.8,27,20.0625,0.0,6.2,0.0,0.0,3.9,-2.7
4,2015-01-01,New York,None,0.4,NaN,0.2875,None,5.3,7.0,3.0625,...,None,29.8,27,20.0625,0.0,NaN,NaN,0.0,NaN,NaN


In [295]:
epa_noaa_df.to_csv('epa_noaa.csv')